In [0]:
%pip install dbldatagen
%pip install Faker

Python interpreter will be restarted.
Python interpreter will be restarted.
Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
from dbldatagen import DataGenerator , PyfuncText
from faker import Faker
from datetime import timedelta, datetime
from pyspark.sql import SparkSession
from pyspark.sql.functions import count, when, isnan, col, lit, countDistinct,substring,split
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, TimestampType
from pyspark.sql.functions import concat_ws,col
import random as random

In [0]:
schema= StructType([
    StructField("user_id",StringType(),False),
    StructField("email_id",StringType(),True),
    StructField("first_name",StringType(),True),
    StructField("last_name",StringType(),True),
    StructField("phone_number", StringType(),True),
    StructField("user_type",StringType(),True),
    StructField("updated_at",StringType(),True)

])

In [0]:
fake=Faker(locale='en_IN')
email_id=(lambda context,v: fake.email())
first_name=(lambda context,v: fake.first_name())
last_name=(lambda context,v: fake.last_name())
phone_number=(lambda context,v: fake.phone_number())
user_type=(lambda context,v: random.choice(['member','non member']))
updated_at=(lambda context,v: fake.date_between(start_date='-15d', end_date='today').strftime('%Y-%m-%d'))

membership_data=(DataGenerator(spark,rows=10,partition=2)
                .withSchema(schema)
                .withColumnSpec('user_id',minValue=1,maxValue=1000,step=1)
                .withColumnSpec('email_id',text=PyfuncText(email_id))
                .withColumnSpec('first_name',text=PyfuncText(first_name))
                .withColumnSpec('last_name',text=PyfuncText(last_name))
                .withColumnSpec('phone_number',text=PyfuncText(phone_number))
                .withColumnSpec('user_type',text=PyfuncText(user_type))
                .withColumnSpec('updated_at',text=PyfuncText(updated_at))
)

databuild=membership_data.build()
display(databuild)


user_id,email_id,first_name,last_name,phone_number,user_type,updated_at
1,ggulati@example.org,Alisha,Kar,06108300693,member,2023-09-29
2,ggulati@example.org,Alisha,Kar,06108300693,member,2023-09-29
3,ggulati@example.org,Alisha,Kar,06108300693,non member,2023-09-29
4,ggulati@example.org,Alisha,Kar,06108300693,member,2023-09-29
5,badalbabu@example.com,Uthkarsh,Gulati,07604608720,non member,2023-09-28
6,ggulati@example.org,Alisha,Kar,06108300693,non member,2023-09-29
7,ggulati@example.org,Alisha,Kar,06108300693,non member,2023-09-29
8,ggulati@example.org,Alisha,Kar,06108300693,non member,2023-09-29
9,ggulati@example.org,Alisha,Kar,06108300693,member,2023-09-29
10,badalbabu@example.com,Uthkarsh,Gulati,07604608720,member,2023-09-28


In [0]:
databuild.write.mode('overwrite').format("parquet").saveAsTable("membership_user")